## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

SSLError: [SSL: DECRYPTION_FAILED_OR_BAD_RECORD_MAC] decryption failed or bad record mac (_ssl.c:2580)

In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(["EIN","NAME"],axis=1)
application_df.head()

In [ ]:
# Determine the number of unique values in each column.
#  YOUR CODE GOES HERE
application_df.nunique()

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
application_df["APPLICATION_TYPE"].value_counts()

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ["T9","T13","T12","T2","T25","T14","T29","T15","T17"]

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

In [ ]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
value_counts_classification = application_df['CLASSIFICATION'].value_counts()
value_counts_classification_filtered = value_counts_classification[value_counts_classification>1]
value_counts_classification_filtered 

In [ ]:
classifications_to_replace = list(application_df.CLASSIFICATION.value_counts()[application_df.CLASSIFICATION.value_counts() < 1883].index)
print(classifications_to_replace)

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
#  YOUR CODE GOES HERE
classifications_to_replace = list(application_df.CLASSIFICATION.value_counts()[application_df.CLASSIFICATION.value_counts() < 100].index)
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)
application_df.head()

In [ ]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"],axis=1).values


# Split the preprocessed data into a training and testing dataset
X_train, x_test, Y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(x_test)

## Compile, Train and Evaluate the Model

### 4 Layers with 80 ,30 , 30 , 30 Neurons. Input and hidden layer Activation function is relu. Out put activation finction in Segmoid. 

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 30
hidden_nodes_layer4 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)
# Second hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))
# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))
# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))
# Output layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
# Check the structure of the model
nn.summary()

In [14]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
# Train the model
fit_model = nn.fit(X_train_scaled,Y_train,epochs=100)

Epoch 1/100
858/858 [==============================] - 6s 4ms/step - loss: 0.5667 - accuracy: 0.7214
Epoch 2/100
858/858 [==============================] - 3s 4ms/step - loss: 0.5524 - accuracy: 0.7302
Epoch 3/100
858/858 [==============================] - 3s 4ms/step - loss: 0.5482 - accuracy: 0.7319
Epoch 4/100
858/858 [==============================] - 3s 4ms/step - loss: 0.5473 - accuracy: 0.7308
Epoch 5/100
858/858 [==============================] - 3s 3ms/step - loss: 0.5456 - accuracy: 0.7328
Epoch 6/100
858/858 [==============================] - 3s 3ms/step - loss: 0.5448 - accuracy: 0.7337
Epoch 7/100
858/858 [==============================] - 3s 3ms/step - loss: 0.5436 - accuracy: 0.7342
Epoch 8/100
858/858 [==============================] - 3s 3ms/step - loss: 0.5434 - accuracy: 0.7344
Epoch 9/100
858/858 [==============================] - 3s 4ms/step - loss: 0.5429 - accuracy: 0.7347
Epoch 10/100
858/858 [==============================] - 3s 4ms/step - loss: 0.5423 - accura

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 1s - loss: 0.5642 - accuracy: 0.7268 - 917ms/epoch - 4ms/step
Loss: 0.5642052292823792, Accuracy: 0.7268221378326416


In [17]:
# Export our model to HDF5 file
nn.save_weights('AlphabetSoupCharity_optimization_3.h5')